In [ ]:
# Several face detection methods. Running on face1
import numpy as np
import os
import face_recognition as frg
from sklearn.neighbors import KNeighborsClassifier
import re
import math
import mtcnn
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display

def show_face(img_path, location):
    pil_image = Image.open(img_path).convert("RGB")
    (top,right, bottom, left) = location

    draw = ImageDraw.Draw(pil_image)
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
    display(pil_image)

In [ ]:
# dlib get_frontal_face_detector()

import dlib
import cv2
detector = dlib.get_frontal_face_detector()
img_path = 'maskedface/train/00009/002.jpg'
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = detector(gray, 1) # result
#to draw faces on image
for result in faces:
    left = result.left()
    top = result.top()
    right = result.right()
    bottom = result.bottom()
    location = [top,right,bottom, left]
    
    show_face(img_path, location)

In [ ]:
# Haar Cascade
# https://towardsdatascience.com/face-detection-models-which-to-use-and-why-d263e82c302c

import cv2
classifier = cv2.CascadeClassifier('models/haarcascade_frontalface_alt2.xml')
img_path = 'maskedface/train/00009/001.jpg'
img = cv2.imread(img_path)
faces = classifier.detectMultiScale(img)# result
#to draw faces on image
for result in faces:
    x, y, w, h = result
    location = [y, x+w,y+h, x]
    show_face(img_path, location)

In [ ]:
# Frontal face detector of DNN module
# https://towardsdatascience.com/face-detection-models-which-to-use-and-why-d263e82c302c

import cv2
import numpy as np
modelFile = "models/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "models/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
img = cv2.imread('maskedface/train/00013/004.jpg')
h, w = img.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
net.setInput(blob)
faces = net.forward()
#to draw faces on image
for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            location = [y,]
            cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)

In [ ]:
# Compare bounding box from MTCNN and face_recognition
filename = 'knn_examples/train/biden/biden.jpg' # filename is defined above, otherwise uncomment
# load image from file
pixels = plt.imread(filename) # defined above, otherwise uncomment
# detector is defined above, otherwise uncomment
detector = mtcnn.MTCNN()
# detect faces in the image
faces = detector.detect_faces(pixels)
print('len(faces):',len(faces))
for result in faces:
    # get coordinates
    x, y, w, h = result['box']
    print('x, y, w, h:',x, y, w, h)
    m_location = [y, x+w,y+h, x]
    print('m_locations:',m_location)
        
f_locations = frg.face_locations(pixels, model='hog')
# display faces on the original image

print('f_locations:',f_locations)
#draw_facebox(filename, faces)
show_face(filename, m_location)